# Assignment 3

In [1]:
import pandas as pd
import numpy as np

# Load MovieLens 100K dataset into a dataframe of pandas
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# Select 500 most active users and 500 most active items from the dataset
n_most_active_users = 500
n_most_active_items = 500

user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index
df = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]

In [3]:
# Map new internal ID for items
i_ids = df['item_id'].unique().tolist()
item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))
df['item_id'] = df['item_id'].map(item_dict)

# Split Dataset

In [4]:
# The number of training users and active users
n_training_users = 300
n_active_users = n_most_active_users - n_training_users

# The number of GIVEN ratings for active users
GIVEN = 20

# Randomly select users from the most active users as training set
random_uids = np.random.choice(df.user_id.unique(), n_training_users, replace=False)
train_df = df[df['user_id'].isin(random_uids)]
# Map new internal ID for all users in the training set
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df['user_id'] = train_df['user_id'].map(user_dict)

# The rest of users are active users for testing
remain_df = df[~df['user_id'].isin(random_uids)]
# Map new internal ID for all active users
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df['user_id'] = remain_df['user_id'].map(user_dict)

# Randomly select GIVEN ratings for active users
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)

test_df = remain_df[~remain_df.index.isin(active_df.index)]

<ipython-input-4-fb140fbf3b12>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['user_id'] = train_df['user_id'].map(user_dict)
<ipython-input-4-fb140fbf3b12>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remain_df['user_id'] = remain_df['user_id'].map(user_dict)


In [5]:
# Convert the format of datasets to matrices
df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_training_users), 'rating': 0})
train_ds = df_zeros.merge(train_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_active_users), 'rating': 0})
active_ds = df_zeros.merge(active_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')
test_ds = df_zeros.merge(test_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

train_ds, active_ds, test_ds

(item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
 user_id                                                    ...                  
 0        3.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 1        0.0  2.0  0.0  4.0  0.0  4.0  4.0  0.0  0.0  2.0  ...  0.0  4.0  4.0   
 2        4.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 3        4.0  0.0  5.0  0.0  0.0  3.0  4.0  2.0  0.0  2.0  ...  0.0  2.0  0.0   
 4        4.0  0.0  5.0  0.0  1.0  0.0  3.0  2.0  0.0  0.0  ...  0.0  0.0  0.0   
 ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 295      4.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  4.0   
 296      4.0  0.0  4.0  0.0  0.0  4.0  3.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 297      0.0  0.0  5.0  4.0  0.0  1.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
 298      4.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 299      0.0  1

In [6]:
# Predicting All Missing Data in training set
imputed_train_ds = train_ds.values.copy()

# Your implementation to predict the missing values
(Put all your implementation for your algorithm in the following cell only to handle the missing values; )

In [8]:
## Put all your implementation for your solutioin in this cell only to predict the missing values; 
## NOTE 1: DO NOT change anything in the rest of the cells in this framework, 
## otherwise the changes might cause errors and make your implementation invalid.

## Note 2: 
## The user-item rating matrix is imputed_train_ds, 
## and the missing values are those 0s in imputed_train_ds. 
## You are required to predict them by using the solution in the given report. 

## The following parameters are required in the given report, 
## which is named "Effective Missing Data Prediction for Collaborative Filtering", 
## and you will need to use them. But, please do not change their values. 
LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ITA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9
# finding the similarity for item - item
item_item_sim_matrix = np.zeros((imputed_train_ds.shape[1], imputed_train_ds.shape[1]))

for i, i_item_vec in enumerate(imputed_train_ds.T):
    for j, j_item_vec in enumerate(imputed_train_ds.T):

        i_mask = i_item_vec > 0
        j_mask = j_item_vec > 0

        corrated_index = np.intersect1d(np.where(i_mask), np.where(j_mask))
        if len(corrated_index) == 0:
            continue

        i_item_mean = np.sum(i_item_vec) / (np.sum(np.clip(i_item_vec, 0, 1)) + EPSILON)
        j_item_mean = np.sum(j_item_vec) / (np.sum(np.clip(j_item_vec, 0, 1)) + EPSILON)

        i_mean_item_sub = i_item_vec[corrated_index] - i_item_mean
        j_mean_item_sub = j_item_vec[corrated_index] - j_item_mean

        r_ui_sub_ri_sq = np.square(i_mean_item_sub)
        r_uj_sub_rj_sq = np.square(j_mean_item_sub)

        avg_sqrt_first_item = np.sqrt(np.sum(r_ui_sub_ri_sq))
        avg_sqrt_second_item = np.sqrt(np.sum(r_uj_sub_rj_sq))

        sim = np.sum(i_mean_item_sub * j_mean_item_sub) / (avg_sqrt_first_item * avg_sqrt_second_item + EPSILON)

        simlarity_weight = (min(len(corrated_index), DELTA) / DELTA) * sim
        
        
        item_item_sim_matrix[i][j] = simlarity_weight

        

#finding similarity for users


user_user_sim_matrix = np.zeros((imputed_train_ds.shape[0], imputed_train_ds.shape[0]))

for i, i_user_vec in enumerate(imputed_train_ds):
    for j, j_user_vec in enumerate(imputed_train_ds):

        i_mask = i_user_vec > 0
        j_mask = j_user_vec > 0

        corrated_index = np.intersect1d(np.where(i_mask), np.where(j_mask))
        if len(corrated_index) == 0:
            continue

        i_user_mean = np.sum(i_user_vec) / (np.sum(np.clip(i_user_vec, 0, 1)) + EPSILON)
        j_user_mean = np.sum(j_user_vec) / (np.sum(np.clip(j_user_vec, 0, 1)) + EPSILON)

        i_mean_user_sub = i_user_vec[corrated_index] - i_user_mean
        j_mean_user_sub = j_user_vec[corrated_index] - j_user_mean

        r_ui_sub_ri_sq = np.square(i_mean_user_sub)
        r_uj_sub_rj_sq = np.square(j_mean_user_sub)

        avg_sqrt_first_user = np.sqrt(np.sum(r_ui_sub_ri_sq))
        avg_sqrt_second_user = np.sqrt(np.sum(r_uj_sub_rj_sq))

        sim = np.sum(i_mean_user_sub * j_mean_user_sub) / (avg_sqrt_first_user * avg_sqrt_second_user + EPSILON)

        simlarity_weight = (min(len(corrated_index), GAMMA) / GAMMA) * sim
        
        user_user_sim_matrix[i][j] =  simlarity_weight

np_predictions = np.zeros((imputed_train_ds.shape[0], imputed_train_ds.shape[1]))


for (i, j), rating in np.ndenumerate(imputed_train_ds):
    
         
    #user-item

        #find the similar users
        sim_user_ids = np.argsort(user_user_sim_matrix[i])
        #similarity co-efficent of values of user 
        sim_value = []
        sim_user_ids_1 = []
        for m in range(len(user_user_sim_matrix[i][sim_user_ids])):
            
            
            
            if user_user_sim_matrix[i][sim_user_ids][m] > ITA and  i != sim_user_ids[m]:

                sim_value.append(user_user_sim_matrix[i][sim_user_ids][m])
                sim_user_ids_1.append(sim_user_ids[m])
        sim_value = np.array(sim_value)
        sim_user_ids_1 = np.array(sim_user_ids_1)
    
    #item-user
        
        sim_item_ids = np.argsort(item_item_sim_matrix[j])
        sim_item_value =[]
        sim_item_ids_1=[]
        
        for n in range(len(item_item_sim_matrix[j][sim_item_ids])):
                      
            if item_item_sim_matrix[j][sim_item_ids][n] > THETA and j != sim_item_ids[n] :
                
                sim_item_value.append(item_item_sim_matrix[j][sim_item_ids][n])
                sim_item_ids_1.append(sim_item_ids[n])
        
        sim_item_value = np.array(sim_item_value)
        sim_item_ids_1 = np.array(sim_item_ids_1)
       
        if (sim_item_value.size > 0) and (sim_value.size == 0):

            
            sim_items = imputed_train_ds.T[sim_item_ids_1]
            item_mean = np.sum(imputed_train_ds.T[j]) / (np.sum(np.clip(imputed_train_ds.T[j], 0, 1))+EPSILON)
            sim_item_mean = np.sum(sim_items,axis = 1) / (np.sum(np.clip(sim_items, 0 ,1),axis = 1)+EPSILON)
            mask_rated_i = sim_items[:,i] >0
            
            sim_r_sum_mean = sim_item_value[mask_rated_i] * (sim_items[mask_rated_i, i] - sim_item_mean[mask_rated_i])

            np_predictions[i][j] = item_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_item_value[mask_rated_i]) + EPSILON)
            np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
            
            if imputed_train_ds[i][j] == 0:
                imputed_train_ds[i][j] = np_predictions[i][j]
        
        
            
        elif (sim_value.size > 0) and (sim_item_value.size == 0):

          
            sim_users = imputed_train_ds[sim_user_ids_1]
            
            user_mean = np.sum(imputed_train_ds[i]) / (np.sum(np.clip(imputed_train_ds[i], 0, 1))+EPSILON)
            sim_user_mean = np.sum(sim_users,axis = 1) / (np.sum(np.clip(sim_users, 0 ,1),axis = 1)+EPSILON)
            mask_rated_j = sim_users[:,j] >0

            sim_r_sum_mean = sim_value[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])

            np_predictions[i][j] = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_value[mask_rated_j]) + EPSILON)
            np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
            
            if imputed_train_ds[i][j] == 0:
                imputed_train_ds[i][j] = np_predictions[i][j]
            
            
        elif(sim_value.size > 0) and (sim_item_value.size > 0):

            sim_items = imputed_train_ds.T[sim_item_ids_1]
            item_mean = np.sum(imputed_train_ds.T[j]) / (np.sum(np.clip(imputed_train_ds.T[j], 0, 1))+EPSILON)
            sim_item_mean = np.sum(sim_items,axis = 1) / (np.sum(np.clip(sim_items, 0 ,1),axis = 1)+EPSILON)
            mask_rated_i = sim_items[:,i] >0
            
            sim_r_sum_mean = sim_item_value[mask_rated_i] * (sim_items[mask_rated_i, i] - sim_item_mean[mask_rated_i])

            item = item_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_item_value[mask_rated_i]) + EPSILON)
            
            
            
            sim_users = imputed_train_ds[sim_user_ids_1]
            user_mean = np.sum(imputed_train_ds[i]) / (np.sum(np.clip(imputed_train_ds[i], 0, 1))+EPSILON)
            sim_user_mean = np.sum(sim_users,axis = 1) / (np.sum(np.clip(sim_users, 0 ,1),axis = 1)+EPSILON)
            mask_rated_j = sim_users[:,j] >0

            sim_r_sum_mean = sim_value[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])

            user = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_value[mask_rated_j]) + EPSILON)
            
            np_predictions[i][j] = ((LAMBDA)*(user) )+ ((1 - LAMBDA)*(item))
            
            np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
            
            if imputed_train_ds[i][j] == 0:
                imputed_train_ds[i][j] = np_predictions[i][j]
                
        elif (sim_item_value.size == 0) and (sim_value.size == 0):
           

            np_predictions[i][j] = 0
            if imputed_train_ds[i][j] == 0:
             
                imputed_train_ds[i][j] = np_predictions[i][j]
                     
    
        


# Evaluation

### Compute Pearson Correlation Coefficient of All Pairs of Items between active set and imputed training set

In [10]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
imputed_train_ds

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,3.000000,4.425000,4.163462,4.868377,3.428571,3.359223,3.945205,4.000000,3.071429,0.000000,...,2.983615,3.215549,0.000000,4.001657,3.159091,2.369024,4.000000,3.734994,3.000000,3.765272
1,4.165049,2.000000,4.269816,4.000000,3.889712,4.000000,4.000000,3.683453,1.407198,2.000000,...,3.356034,4.000000,4.000000,4.077264,3.000000,3.000000,2.486636,3.408085,3.458081,1.063915
2,4.000000,4.039949,4.023759,2.000000,3.806218,3.781369,4.540594,4.115244,3.922270,4.301487,...,3.819447,3.815712,3.875167,3.886105,3.590328,3.799523,3.418350,3.897889,3.690143,3.727548
3,4.000000,3.230270,5.000000,3.867234,2.278275,3.000000,4.000000,2.000000,2.828871,2.000000,...,2.825576,2.000000,0.000000,4.890864,4.019502,2.272958,1.528088,4.000000,2.360943,2.316134
4,4.000000,3.562651,5.000000,3.950674,1.000000,3.309741,3.000000,2.000000,3.053829,0.000000,...,2.319641,2.246831,0.000000,4.000000,1.000000,1.243626,2.505380,3.747227,2.938719,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4.000000,3.689616,4.699789,2.977105,3.644907,2.395867,4.059122,4.000000,3.707895,4.151194,...,3.482702,3.518678,4.000000,4.038509,3.623475,3.153619,2.794235,3.950957,3.424261,3.414364
296,4.000000,4.114188,4.000000,3.898316,3.342470,4.000000,3.000000,4.000000,3.204038,0.000000,...,3.247607,3.376936,0.000000,3.835937,2.508232,2.677870,2.620267,3.498808,2.919909,3.307323
297,4.174991,1.682991,5.000000,4.000000,3.509406,1.000000,3.870486,1.577647,3.096070,1.000000,...,2.243997,2.280138,0.000000,3.618064,2.344571,2.714209,2.620267,3.215039,1.000000,2.799744
298,4.000000,3.498013,3.000000,4.200582,3.439198,3.595861,3.740630,3.418045,3.105440,3.885664,...,3.584205,3.596090,4.069438,3.596514,3.224947,3.260443,2.390292,3.527363,3.289727,3.612090


In [11]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

# Compute Pearson Correlation Coefficient of All Pairs of Users between active set and imputed training set
for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair od users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        active_user_pearson_corr[i][j] = weighted_sim

active_user_pearson_corr

array([[ 0.27870428,  0.21009747,  0.19624044, ...,  0.23483853,
         0.46313875, -0.02645221],
       [ 0.41051784,  0.31254818, -0.19580341, ...,  0.49127365,
        -0.10673058, -0.16457144],
       [ 0.41779895,  0.1962339 ,  0.06972013, ...,  0.03678096,
         0.25407147, -0.22907878],
       ...,
       [ 0.12983891,  0.20102846,  0.02875942, ...,  0.23311487,
        -0.16405897,  0.50953562],
       [ 0.25236375,  0.40221228, -0.24685285, ...,  0.27150237,
        -0.07528461,  0.21236752],
       [ 0.12109299,  0.28429491,  0.03330163, ...,  0.31987303,
        -0.16065512,  0.03956353]])

## Predict Ratings of Testing Set

In [12]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        # the coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        user_based_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        user_based_pred = np.clip(user_based_pred, 0, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Compute MAE and RMSE

In [13]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

MAE: 0.7581002492126715, RMSE: 0.9706972723032563
